In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Set display options (corrected)
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)    # Show all rows

# Configure warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
# plotly
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.express as px
import math 
import calendar

In [3]:
os.chdir(r'C:\Users\bhans\Desktop\PROJECTS\accident risk predictor\Data')



In [4]:
import os

# Check if file exists and is readable
if os.path.exists("train.csv"):
    print("File exists!")
    if os.access("train.csv", os.R_OK):
        print("File is readable!")
    else:
        print("No read permissions!")
else:
    print("File not found!")
    

File exists!
File is readable!


In [5]:
train = pd.read_csv(r"C:\Users\bhans\Desktop\PROJECTS\accident risk predictor\Data\train.csv\train.csv")
test = pd.read_csv(r"C:\Users\bhans\Desktop\PROJECTS\accident risk predictor\Data\test.csv\test.csv")
sample_submission= pd.read_csv(r"C:\Users\bhans\Desktop\PROJECTS\accident risk predictor\Data\sample_submission.csv")

In [6]:
train.head(10)


,Accident_ID,Police_Force,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,state,postcode,country
0,1,34,2,1,19/12/12,7,13:20,344,E10000032,4,395,Single carriageway,30,-1,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,Ol or diesel,NaN,1,Yes,England,OX3 9UP,United Kingdom
1,2,5,2,1,02/11/12,4,7:53,102,E09000026,3,13,One way street,30,-1,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Raining without high winds,Dry,NaN,NaN,1,No,England,S35 4EZ,United Kingdom
2,3,1,2,1,02/11/12,4,16:00,531,E10000016,6,8,Roundabout,40,6,0,None within 50 metres,Zebra crossing,Daylight: Street light present,Fine without high winds,Dry,NaN,NaN,1,No,England,BN21 2XR,United Kingdom
3,4,1,1,1,06/05/12,1,16:50,7,E08000035,6,13,Single carriageway,30,6,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,Roadworks,NaN,1,Yes,England,TA20 3PT,United Kingdom
4,5,46,1,1,30/06/12,3,13:25,519,E10000031,3,24,Dual carriageway,30,6,0,None within 50 metres,Zebra crossing,Daylight: Street light present,Fine without high winds,Dry,NaN,NaN,1,No,England,DN20 0QF,United Kingdom
5,6,44,2,1,04/03/12,1,12:31,638,W06000004,6,1,Single carriageway,30,6,0,None within 50 metres,No physical crossing within 50 meters,Darkness: Street lighting unknown,Raining without high winds,Dry,NaN,NaN,1,Yes,England,OL11 1NH,United Kingdom
6,7,43,2,1,11/04/12,2,14:44,502,E06000049,5,4247,Roundabout,30,6,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,NaN,NaN,1,Yes,England,E5 9QH,United Kingdom
7,8,43,2,1,06/06/12,2,21:43,529,S12000019,3,406,Single carriageway,30,5,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,Roadworks,NaN,2,Yes,England,SO14 0FD,United Kingdom
8,9,42,2,3,27/03/12,6,13:30,489,E10000016,4,4,Roundabout,30,6,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,NaN,NaN,1,Yes,England,WS9 8BD,United Kingdom
9,10,6,2,1,13/09/12,6,17:25,95,E06000050,4,12,Single carriageway,30,6,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,NaN,NaN,1,Yes,England,LA4 4JN,United Kingdom


In [7]:
print(train.shape)

(478741, 27)


In [8]:
print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in test'.format(test.shape[0],test.shape[1]))
print('There are {} rows and {} columns in sample_submission'.format(sample_submission.shape[0],sample_submission.shape[1]))

There are 478741 rows and 27 columns in train
There are 121259 rows and 27 columns in test
There are 49772 rows and 2 columns in sample_submission


In [9]:
#Removes train rows which has Local_Authority_(Highway) not present in test set
#test_user_id_list=test['Local_Authority_(Highway)'].tolist()
#train=train[train['Local_Authority_(Highway)'].isin(test_user_id_list)]

In [10]:
#Check unique values of each column
'''for i in train.columns.drop(['Accident_ID','Date','Time']):
    print(i,np.sort(train[i].unique()))'''

"for i in train.columns.drop(['Accident_ID','Date','Time']):\n    print(i,np.sort(train[i].unique()))"

In [11]:
#Handle null values
#train.dropna(axis=0,inplace=True)
#test.Time.fillna(method='ffill',inplace=True)

#train["Time"]=train["Time"].fillna(train["Time"].mode()[0])
train["Time"]=train["Time"].fillna("00:00")

train["Road_Surface_Conditions"]=train["Road_Surface_Conditions"].fillna(train["Road_Surface_Conditions"].mode()[0])
train["Special_Conditions_at_Site"]=train["Special_Conditions_at_Site"].fillna(train["Special_Conditions_at_Site"].mode()[0])

#test["Time"]=test["Time"].fillna(test["Time"].mode()[0])
test["Time"]=test["Time"].fillna("00:00")
test["Road_Surface_Conditions"]=test["Road_Surface_Conditions"].fillna(test["Road_Surface_Conditions"].mode()[0])
test["Special_Conditions_at_Site"]=test["Special_Conditions_at_Site"].fillna(test["Special_Conditions_at_Site"].mode()[0])

In [12]:
#Convert to datetime
train['Date'] = train['Date'] + " " + train['Time']
train['Date'] =  pd.to_datetime(train['Date'], format='%d/%m/%y %H:%M')

test['Date'] = test['Date'] + " " + test['Time']
test['Date'] =  pd.to_datetime(test['Date'], format='%d/%m/%y %H:%M')

#train
train["day_in_month"]=train.Date.dt.day
train["month_in_year"]=train.Date.dt.month
train["year"]=train.Date.dt.year
train["hour"] = train["Date"].dt.hour
train['dayofweek'] = train['Date'].dt.dayofweek
train['Minute']=train['hour']*60.0+train["Date"].dt.minute
train['WeekofYear'] = train['Date'].apply(lambda x : x.weekofyear)

#test
test["day_in_month"]=test.Date.dt.day
test["month_in_year"]=test.Date.dt.month
test["year"]=test.Date.dt.year
test["hour"] = test["Date"].dt.hour
test['dayofweek'] = test['Date'].dt.dayofweek
test['Minute']=test['hour']*60.0+test["Date"].dt.minute
test['WeekofYear'] = test['Date'].apply(lambda x : x.weekofyear)


df1=test.copy()

train.head(3)

,Accident_ID,Police_Force,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,state,postcode,country,day_in_month,month_in_year,year,hour,dayofweek,Minute,WeekofYear
0,1,34,2,1,2012-12-19 13:20:00,7,13:20,344,E10000032,4,395,Single carriageway,30,-1,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,Ol or diesel,NaN,1,Yes,England,OX3 9UP,United Kingdom,19,12,2012,13,2,800.0,51
1,2,5,2,1,2012-11-02 07:53:00,4,7:53,102,E09000026,3,13,One way street,30,-1,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Raining without high winds,Dry,Roadworks,NaN,1,No,England,S35 4EZ,United Kingdom,2,11,2012,7,4,473.0,44
2,3,1,2,1,2012-11-02 16:00:00,4,16:00,531,E10000016,6,8,Roundabout,40,6,0,None within 50 metres,Zebra crossing,Daylight: Street light present,Fine without high winds,Dry,Roadworks,NaN,1,No,England,BN21 2XR,United Kingdom,2,11,2012,16,4,960.0,44


In [13]:
### Try New starts ###

In [14]:
train["Number_of_Casualties"].value_counts()

Number_of_Casualties
1    329124
2     98814
3     35399
4      9318
5      6086
Name: count, dtype: int64

In [15]:
print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in test'.format(test.shape[0],test.shape[1]))

There are 478741 rows and 34 columns in train
There are 121259 rows and 34 columns in test


In [16]:
train['train_or_test']='train'
test['train_or_test']='test'

df=pd.concat([train,test])

print("Combined dataset shape: {0}: ".format(df.shape))

Combined dataset shape: (600000, 35): 


In [17]:
df["maximum_hours"] = df["hour"].isin([8, 12,13,14,15,16,17,18]).astype("object")

In [18]:
# frequence encoding and log-transform
df['Minute_Freq'] = df.groupby(['Minute'])['Minute'].transform('count')
df['Minute_Freq'] = df['Minute_Freq']/df.shape[0]*24*60
df['Minute_Freq'] = df['Minute_Freq'].apply(lambda x: np.log(x+1))

In [19]:
print("Combined dataset shape: {0}: ".format(df.shape))

Combined dataset shape: (600000, 37): 


In [20]:
import category_encoders as ce
#postcode
cat_features = ['postcode']

# Create the encoder
count_enc = ce.CountEncoder()

# Transform the features, rename the columns with the _count suffix, and join to dataframe
df['postcode_cnt'] = count_enc.fit_transform(df[cat_features])

#Local_Authority_(Highway)
cat_features_1 = ['Local_Authority_(Highway)']

# Create the encoder
count_enc = ce.CountEncoder()

# Transform the features, rename the columns with the _count suffix, and join to dataframe
df['LAH_cnt'] = count_enc.fit_transform(df[cat_features_1])

#Local_Authority_(District)	
cat_features_2 = ['Local_Authority_(District)']

# Create the encoder
count_enc = ce.CountEncoder()

# Transform the features, rename the columns with the _count suffix, and join to dataframe
df['LAD_cnt'] = count_enc.fit_transform(df[cat_features_2])

#Police_Force
cat_features_3 = ['Police_Force']

# Create the encoder
count_enc = ce.CountEncoder()

# Transform the features, rename the columns with the _count suffix, and join to dataframe
df['PF_cnt'] = count_enc.fit_transform(df[cat_features_3])

In [21]:
print("Combined dataset shape: {0}: ".format(df.shape))

Combined dataset shape: (600000, 41): 


In [22]:
df.sample(3)

,Accident_ID,Police_Force,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,state,postcode,country,day_in_month,month_in_year,year,hour,dayofweek,Minute,WeekofYear,train_or_test,maximum_hours,Minute_Freq,postcode_cnt,LAH_cnt,LAD_cnt,PF_cnt
389875,488648,4,2,1,2012-09-12 09:00:00,6,9:00,90,E10000016,6,14,Roundabout,30,6,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,Roadworks,NaN,1,Yes,England,MK45 2PL,United Kingdom,12,9,2012,9,2,540.0,37,train,False,1.881844,12,33979,90,4
469228,588108,43,2,2,2012-09-12 17:15:00,4,17:15,679,E08000032,3,16,Single carriageway,70,3,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,Roadworks,NaN,1,Yes,England,PO2 9RS,United Kingdom,12,9,2012,17,2,1035.0,37,train,True,2.268656,4,19424,679,43
65742,82267,33,3,1,2012-02-07 18:15:00,6,18:15,299,E10000002,6,11,Single carriageway,40,6,0,None within 50 metres,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,Roadworks,NaN,2,No,England,BS24 0AW,United Kingdom,7,2,2012,18,1,1095.0,6,train,True,2.054944,6,4253,299,33


In [23]:
#creating function to identify time of day: morning, afternoon, evening, night, etc.
def time_of_day(n):
    if n in range(4,8):
        return 'Early Morning'
    elif n in range(8,12):
        return 'Morning'
    elif n in range(12,17):
        return 'Afternoon'
    elif n in range(17,20):
        return 'Evening'
    elif n in range(20,25) or n==0:
        return 'Night'
    elif n in range(1,4):
        return 'Late Night'
    
df['Type_of_Day']=df['hour'].apply(lambda x: time_of_day(x))

In [24]:
def month2seasons(x):
    if x in [9, 10, 11]:
        season = 'Spring'
    elif x in [12, 1, 2]:
        season = 'Summer'
    elif x in [3, 4, 5]:
        season = 'Autumn'
    elif x in [6, 7, 8]:
        season = 'Winter'
    return season

df['Season'] = df['month_in_year'].apply(month2seasons)
df.head(3)

,Accident_ID,Police_Force,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,state,postcode,country,day_in_month,month_in_year,year,hour,dayofweek,Minute,WeekofYear,train_or_test,maximum_hours,Minute_Freq,postcode_cnt,LAH_cnt,LAD_cnt,PF_cnt,Type_of_Day,Season
0,1,34,2,1,2012-12-19 13:20:00,7,13:20,344,E10000032,4,395,Single carriageway,30,-1,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,Ol or diesel,NaN,1,Yes,England,OX3 9UP,United Kingdom,19,12,2012,13,2,800.0,51,train,True,2.638429,7,18011,344,34,Afternoon,Summer
1,2,5,2,1,2012-11-02 07:53:00,4,7:53,102,E09000026,3,13,One way street,30,-1,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Raining without high winds,Dry,Roadworks,NaN,1,No,England,S35 4EZ,United Kingdom,2,11,2012,7,4,473.0,44,train,False,0.430223,12,2681,102,5,Early Morning,Spring
2,3,1,2,1,2012-11-02 16:00:00,4,16:00,531,E10000016,6,8,Roundabout,40,6,0,None within 50 metres,Zebra crossing,Daylight: Street light present,Fine without high winds,Dry,Roadworks,NaN,1,No,England,BN21 2XR,United Kingdom,2,11,2012,16,4,960.0,44,train,True,3.002112,14,33979,531,1,Afternoon,Spring


# # Feature Eng starts

In [25]:
from itertools import combinations
from sklearn.preprocessing import LabelEncoder

def frequency_encoding(column_name,output_column_name,df):
    fe_pol = (df.groupby(column_name).size()) / len(df)
    df[output_column_name] = df[column_name].apply(lambda x : fe_pol[x])

le = LabelEncoder()
le_features=[]
cat_features=[]

columns=['Police_Force','1st_Road_Class','1st_Road_Number','2nd_Road_Class','Speed_limit','Local_Authority_(District)','Local_Authority_(Highway)','Number_of_Vehicles','Urban_or_Rural_Area','Road_Type']


comb = combinations(columns, 2) 

for i in list(comb):  
    df[f'{i[0]}_{i[1]}']=df[i[0]].astype(str)+'_'+df[i[1]].astype(str)
    df[f'{i[0]}_{i[1]}_le']=le.fit_transform(df[f'{i[0]}_{i[1]}'])
    frequency_encoding(f'{i[0]}_{i[1]}',f'{i[0]}_{i[1]}',df)
    cat_features.append(f'{i[0]}_{i[1]}')  

frequency_encoding('1st_Road_Class','1st_Road_Class_fe',df)
frequency_encoding('1st_Road_Number','1st_Road_Number_fe',df)
frequency_encoding('2nd_Road_Class','2nd_Road_Class_fe',df)
frequency_encoding('Speed_limit','Speed_limit_fe',df)
frequency_encoding('Urban_or_Rural_Area','Urban_or_Rural_Area_fe',df)
frequency_encoding('Number_of_Vehicles','Number_of_Vehicles_fe',df)
frequency_encoding('Road_Type','Road_Type_fe',df)

'''Road_Type_aggregate_features = df.groupby(['Road_Type']).agg({'Speed_limit': ['mean', 'sum'],
                                                      
                                                     })
Road_Type_aggregate_features.columns = ['Road_Type_aggregate_features' + '_'.join(c).strip('_') for c in Road_Type_aggregate_features.columns]
df = pd.merge(df, Road_Type_aggregate_features, on = ['Road_Type'], how='left')'''

C:\Users\bhans\AppData\Local\Temp\ipykernel_20652\3736814143.py:19: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\bhans\AppData\Local\Temp\ipykernel_20652\3736814143.py:18: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\bhans\AppData\Local\Temp\ipykernel_20652\3736814143.py:19: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

"Road_Type_aggregate_features = df.groupby(['Road_Type']).agg({'Speed_limit': ['mean', 'sum'],\n\n                                                     })\nRoad_Type_aggregate_features.columns = ['Road_Type_aggregate_features' + '_'.join(c).strip('_') for c in Road_Type_aggregate_features.columns]\ndf = pd.merge(df, Road_Type_aggregate_features, on = ['Road_Type'], how='left')"

In [26]:
df.sample(15)

,Accident_ID,Police_Force,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,state,postcode,country,day_in_month,month_in_year,year,hour,dayofweek,Minute,WeekofYear,train_or_test,maximum_hours,Minute_Freq,postcode_cnt,LAH_cnt,LAD_cnt,PF_cnt,Type_of_Day,Season,Police_Force_1st_Road_Class,Police_Force_1st_Road_Class_le,Police_Force_1st_Road_Number,Police_Force_1st_Road_Number_le,Police_Force_2nd_Road_Class,Police_Force_2nd_Road_Class_le,Police_Force_Speed_limit,Police_Force_Speed_limit_le,Police_Force_Local_Authority_(District),Police_Force_Local_Authority_(District)_le,Police_Force_Local_Authority_(Highway),Police_Force_Local_Authority_(Highway)_le,Police_Force_Number_of_Vehicles,Police_Force_Number_of_Vehicles_le,Police_Force_Urban_or_Rural_Area,Police_Force_Urban_or_Rural_Area_le,Police_Force_Road_Type,Police_Force_Road_Type_le,1st_Road_Class_1st_Road_Number,1st_Road_Class_1st_Road_Number_le,1st_Road_Class_2nd_Road_Class,1st_Road_Class_2nd_Road_Class_le,1st_Road_Class_Speed_limit,1st_Road_Class_Speed_limit_le,1st_Road_Class_Local_Authority_(District),1st_Road_Class_Local_Authority_(District)_le,1st_Road_Class_Local_Authority_(Highway),1st_Road_Class_Local_Authority_(Highway)_le,1st_Road_Class_Number_of_Vehicles,1st_Road_Class_Number_of_Vehicles_le,1st_Road_Class_Urban_or_Rural_Area,1st_Road_Class_Urban_or_Rural_Area_le,1st_Road_Class_Road_Type,1st_Road_Class_Road_Type_le,1st_Road_Number_2nd_Road_Class,1st_Road_Number_2nd_Road_Class_le,1st_Road_Number_Speed_limit,1st_Road_Number_Speed_limit_le,1st_Road_Number_Local_Authority_(District),1st_Road_Number_Local_Authority_(District)_le,1st_Road_Number_Local_Authority_(Highway),1st_Road_Number_Local_Authority_(Highway)_le,1st_Road_Number_Number_of_Vehicles,1st_Road_Number_Number_of_Vehicles_le,1st_Road_Number_Urban_or_Rural_Area,1st_Road_Number_Urban_or_Rural_Area_le,1st_Road_Number_Road_Type,1st_Road_Number_Road_Type_le,2nd_Road_Class_Speed_limit,2nd_Road_Class_Speed_limit_le,2nd_Road_Class_Local_Authority_(District),2nd_Road_Class_Local_Authority_(District)_le,2nd_Road_Class_Local_Authority_(Highway),2nd_Road_Class_Local_Authority_(Highway)_le,2nd_Road_Class_Number_of_Vehicles,2nd_Road_Class_Number_of_Vehicles_le,2nd_Road_Class_Urban_or_Rural_Area,2nd_Road_Class_Urban_or_Rural_Area_le,2nd_Road_Class_Road_Type,2nd_Road_Class_Road_Type_le,Speed_limit_Local_Authority_(District),Speed_limit_Local_Authority_(District)_le,Speed_limit_Local_Authority_(Highway),Speed_limit_Local_Authority_(Highway)_le,Speed_limit_Number_of_Vehicles,Speed_limit_Number_of_Vehicles_le,Speed_limit_Urban_or_Rural_Area,Speed_limit_Urban_or_Rural_Area_le,Speed_limit_Road_Type,Speed_limit_Road_Type_le,Local_Authority_(District)_Local_Authority_(Highway),Local_Authority_(District)_Local_Authority_(Highway)_le,Local_Authority_(District)_Number_of_Vehicles,Local_Authority_(District)_Number_of_Vehicles_le,Local_Authority_(District)_Urban_or_Rural_Area,Local_Authority_(District)_Urban_or_Rural_Area_le,Local_Authority_(District)_Road_Type,Local_Authority_(District)_Road_Type_le,Local_Authority_(Highway)_Number_of_Vehicles,Local_Authority_(Highway)_Number_of_Vehicles_le,Local_Authority_(Highway)_Urban_or_Rural_Area,Local_Authority_(Highway)_Urban_or_Rural_Area_le,Local_Authority_(Highway)_Road_Type,Local_Authority_(Highway)_Road_Type_le,Number_of_Vehicles_Urban_or_Rural_Area,Number_of_Vehicles_Urban_or_Rural_Area_le,Number_of_Vehicles_Road_Type,Number_of_Vehicles_Road_Type_le,Urban_or_Rural_Area_Road_Type,Urban_or_Rural_Area_Road_Type_le,1st_Road_Class_fe,1st_Road_Number_fe,2nd_Road_Class_fe,Speed_limit_fe,Urban_or_Rural_Area_fe,Numbe

In [27]:
# Feature Eng ends

In [28]:
print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in test'.format(test.shape[0],test.shape[1]))

There are 478741 rows and 35 columns in train
There are 121259 rows and 35 columns in test


In [29]:
train=df.loc[df.train_or_test.isin(['train'])]
test=df.loc[df.train_or_test.isin(['test'])]

In [30]:
object_columns = ['Number_of_Vehicles','1st_Road_Class','Road_Type','Speed_limit','2nd_Road_Class','Pedestrian_Crossing-Human_Control','Pedestrian_Crossing-Physical_Facilities','Light_Conditions','Weather_Conditions','Road_Surface_Conditions','Special_Conditions_at_Site','Carriageway_Hazards','Did_Police_Officer_Attend_Scene_of_Accident','state','month_in_year','Urban_or_Rural_Area','dayofweek']
train[object_columns] = train[object_columns].astype(object)
test[object_columns] = test[object_columns].astype(object)

C:\Users\bhans\AppData\Local\Temp\ipykernel_20652\397953129.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\bhans\AppData\Local\Temp\ipykernel_20652\397953129.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
#EDA

In [32]:
### Merging starts ###

In [33]:
population = pd.read_csv('population.csv')
population.head(3)

,postcode,Rural Urban,Variable: All usual residents; measures: Value,Variable: Males; measures: Value,Variable: Females; measures: Value,Variable: Lives in a household; measures: Value,Variable: Lives in a communal establishment; measures: Value,Variable: Schoolchild or full-time student aged 4 and over at their non term-time address; measures: Value,Variable: Area (Hectares); measures: Value,Variable: Density (number of persons per hectare); measures: Value
0,AL1 1,Total,5453,2715,2738,5408,45,75,225.63,24.2
1,AL1 2,Total,6523,3183,3340,6418,105,77,286.59,22.8
2,AL1 3,Total,4179,2121,2058,4100,79,46,97.12,43.0


In [34]:
roads_network = pd.read_csv(r'C:\Users\bhans\Desktop\projects_reference\Predict accident risk score for unique postcode\Data\roads_network.csv\roads_network.csv')
roads_network.head()

,WKT,roadClassi,roadFuncti,formOfWay,length,primaryRou,distance to the nearest point on rd,postcode
0,POINT (-2.3501 56.603923),A Road,A Road,Single Carriageway,2643.0,1.0,1.256769,AB1
1,POINT (-2.021334 57.130142),A Road,A Road,Single Carriageway,2643.0,1.0,1.834101,AB1 9NN
2,POINT (-2.108598 57.146338),A Road,A Road,Single Carriageway,2643.0,1.0,1.830243,AB10 1UH
3,POINT (-2.093928 57.148218),A Road,A Road,Single Carriageway,2643.0,1.0,1.835092,AB10 1YL
4,POINT (-2.116089 57.131671),A Road,A Road,Single Carriageway,2643.0,1.0,1.814373,AB10 6AT


In [35]:
train.Road_Type.value_counts()

Road_Type
Single carriageway    372554
Dual carriageway       52783
Roundabout             28541
One way street         24311
Slip road                523
Unknown                   29
Name: count, dtype: int64

In [36]:
roads_network.formOfWay.value_counts()

formOfWay
Single Carriageway            46015
Collapsed Dual Carriageway    22290
Roundabout                    16667
Slip Road                      3469
Dual Carriageway               1911
Name: count, dtype: int64

In [37]:
roads_network["WKT"] = roads_network["WKT"].str.replace('[a-zA-Z]', '')
roads_network["WKT"]= roads_network["WKT"].replace(r'[(]+', ' ', regex=True)
roads_network["WKT"]= roads_network["WKT"].replace(r'[)]+', ' ', regex=True)
roads_network.head()

,WKT,roadClassi,roadFuncti,formOfWay,length,primaryRou,distance to the nearest point on rd,postcode
0,POINT -2.3501 56.603923,A Road,A Road,Single Carriageway,2643.0,1.0,1.256769,AB1
1,POINT -2.021334 57.130142,A Road,A Road,Single Carriageway,2643.0,1.0,1.834101,AB1 9NN
2,POINT -2.108598 57.146338,A Road,A Road,Single Carriageway,2643.0,1.0,1.830243,AB10 1UH
3,POINT -2.093928 57.148218,A Road,A Road,Single Carriageway,2643.0,1.0,1.835092,AB10 1YL
4,POINT -2.116089 57.131671,A Road,A Road,Single Carriageway,2643.0,1.0,1.814373,AB10 6AT


In [38]:
roads_network = roads_network.applymap(lambda x: x.strip() if isinstance(x, str) else x)
roads_network.head()

,WKT,roadClassi,roadFuncti,formOfWay,length,primaryRou,distance to the nearest point on rd,postcode
0,POINT -2.3501 56.603923,A Road,A Road,Single Carriageway,2643.0,1.0,1.256769,AB1
1,POINT -2.021334 57.130142,A Road,A Road,Single Carriageway,2643.0,1.0,1.834101,AB1 9NN
2,POINT -2.108598 57.146338,A Road,A Road,Single Carriageway,2643.0,1.0,1.830243,AB10 1UH
3,POINT -2.093928 57.148218,A Road,A Road,Single Carriageway,2643.0,1.0,1.835092,AB10 1YL
4,POINT -2.116089 57.131671,A Road,A Road,Single Carriageway,2643.0,1.0,1.814373,AB10 6AT


In [39]:
# Split into two columns and assign directly
roads_network[['Longitude', 'Latitude']] = roads_network['WKT'].str.split(' ', n=1, expand=True)
roads_network.sample(3)

,WKT,roadClassi,roadFuncti,formOfWay,length,primaryRou,distance to the nearest point on rd,postcode,Longitude,Latitude
54214,POINT 1.713625 52.57503,A Road,A Road,Single Carriageway,1038.0,1.0,0.068996,NR31 7DD,POINT,1.713625 52.57503
48860,POINT -0.19737 51.606707,A Road,A Road,Collapsed Dual Carriageway,364.0,1.0,0.016900,N3 1EY,POINT,-0.19737 51.606707
84944,POINT -0.309093 51.488032,A Road,A Road,Collapsed Dual Carriageway,219.0,1.0,0.001810,TW8 8DT,POINT,-0.309093 51.488032


In [40]:
print(roads_network['WKT'].head())

0      POINT  -2.3501 56.603923
1    POINT  -2.021334 57.130142
2    POINT  -2.108598 57.146338
3    POINT  -2.093928 57.148218
4    POINT  -2.116089 57.131671
Name: WKT, dtype: object


In [41]:
# First extract the coordinates from parentheses
roads_network[['Longitude', 'Latitude']] = (
    roads_network['WKT']
    .str.extract(r'POINT\(([^ ]+) ([^ ]+)\)')
)

In [42]:
# Convert to Web Mercator coordinate system.
import pyproj as _proj
src_prj = _proj.Proj("+init=EPSG:4326")
dst_prj = _proj.Proj("+init=EPSG:3857")

x_coords = roads_network.Longitude.values
y_coords = roads_network.Latitude.values
x_coords, y_coords = _proj.transform(src_prj, dst_prj, x_coords, y_coords)
roads_network["x"] = x_coords
roads_network["y"] = y_coords

In [43]:
roads_network.head(5)

,WKT,roadClassi,roadFuncti,formOfWay,length,primaryRou,distance to the nearest point on rd,postcode,Longitude,Latitude,x,y
0,POINT -2.3501 56.603923,A Road,A Road,Single Carriageway,2643.0,1.0,1.256769,AB1,NaN,NaN,NaN,NaN
1,POINT -2.021334 57.130142,A Road,A Road,Single Carriageway,2643.0,1.0,1.834101,AB1 9NN,NaN,NaN,NaN,NaN
2,POINT -2.108598 57.146338,A Road,A Road,Single Carriageway,2643.0,1.0,1.830243,AB10 1UH,NaN,NaN,NaN,NaN
3,POINT -2.093928 57.148218,A Road,A Road,Single Carriageway,2643.0,1.0,1.835092,AB10 1YL,NaN,NaN,NaN,NaN
4,POINT -2.116089 57.131671,A Road,A Road,Single Carriageway,2643.0,1.0,1.814373,AB10 6AT,NaN,NaN,NaN,NaN


In [44]:
roads_network.shape

(91566, 12)

In [45]:
roads_network.drop_duplicates(subset=['postcode'], keep='first', inplace=True)

In [46]:
roads_network.shape

(75895, 12)

In [47]:

print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in test'.format(test.shape[0],test.shape[1]))

There are 478741 rows and 140 columns in train
There are 121259 rows and 140 columns in test


In [48]:
train = pd.merge(train, population, on = ['postcode'],how='left')
test  = pd.merge(test, population, on = ['postcode'],how='left')

train = pd.merge(train, roads_network, on = ['postcode'],how='left')
test  = pd.merge(test, roads_network, on = ['postcode'],how='left')

In [49]:
'''k=roads_network[(roads_network["postcode"]=='NE35 9EA')]
k.head(10)'''

'k=roads_network[(roads_network["postcode"]==\'NE35 9EA\')]\nk.head(10)'

In [50]:
print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in test'.format(test.shape[0],test.shape[1]))

There are 478741 rows and 160 columns in train
There are 121259 rows and 160 columns in test


In [51]:
#drop columns
drop_columns_1 = ["Rural Urban","WKT","roadFuncti","formOfWay"]
train.drop(drop_columns_1, axis=1, inplace=True)
test.drop(drop_columns_1, axis=1, inplace=True)

#Replace NAN with "999"
train.replace(to_replace=np.nan, value='999', inplace = True)
test.replace(to_replace=np.nan, value='999', inplace = True)

In [52]:
int_columns = ['Variable: All usual residents; measures: Value',
               'Variable: Males; measures: Value',
               'Variable: Females; measures: Value',
               'Variable: Lives in a household; measures: Value',
               'Variable: Lives in a communal establishment; measures: Value',
               'Variable: Schoolchild or full-time student aged 4 and over at their non term-time address; measures: Value',
               ]

float_columns = ['Variable: Area (Hectares); measures: Value', 
                 'Variable: Density (number of persons per hectare); measures: Value',
                 'distance to the nearest point on rd',
                 'length',
                 'Latitude',
                 'Longitude',
                 'x',
                 'y']

train[int_columns] = train[int_columns].astype(int)
test[int_columns]  = test[int_columns].astype(int)

train[float_columns] = train[float_columns].astype(float)
test[float_columns]  = test[float_columns].astype(float)

In [53]:
train['train_or_test']='train'
test['train_or_test']='test'

df=pd.concat([train,test])

print("Combined dataset shape: {0}: ".format(df.shape))

Combined dataset shape: (600000, 156): 


In [54]:
'''Road_Type_aggregate_features = df.groupby(['Road_Type']).agg({'distance to the nearest point on rd': ['mean', 'sum'],
                                                      
                                                     })
Road_Type_aggregate_features.columns = ['Road_Type_aggregate_features' + '_'.join(c).strip('_') for c in Road_Type_aggregate_features.columns]
df = pd.merge(df, Road_Type_aggregate_features, on = ['Road_Type'], how='left')'''

"Road_Type_aggregate_features = df.groupby(['Road_Type']).agg({'distance to the nearest point on rd': ['mean', 'sum'],\n\n                                                     })\nRoad_Type_aggregate_features.columns = ['Road_Type_aggregate_features' + '_'.join(c).strip('_') for c in Road_Type_aggregate_features.columns]\ndf = pd.merge(df, Road_Type_aggregate_features, on = ['Road_Type'], how='left')"

In [55]:
#Divide by features
#df["length_by_Speed_limit"] = df["length"]/df["Speed_limit"]
#df["length_by_density"] = df["length"]/df["Variable: Density (number of persons per hectare); measures: Value"]
df["density_by_Police_Force"] = df["Variable: Density (number of persons per hectare); measures: Value"]/df["Police_Force"]
df["population_by_Police_Force"] = df["Variable: All usual residents; measures: Value"]/df["Police_Force"]

train=df.loc[df.train_or_test.isin(['train'])]
test=df.loc[df.train_or_test.isin(['test'])]

print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in test'.format(test.shape[0],test.shape[1]))

C:\Users\bhans\AppData\Local\Temp\ipykernel_20652\980073789.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\bhans\AppData\Local\Temp\ipykernel_20652\980073789.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



There are 478741 rows and 158 columns in train
There are 121259 rows and 158 columns in test


In [56]:
### Merging ends ###

In [57]:
train['Date'] = train['Date'].astype('datetime64[ns]').astype(np.int64)/10**9
test['Date'] = test['Date'].astype('datetime64[ns]').astype(np.int64)/10**9

C:\Users\bhans\AppData\Local\Temp\ipykernel_20652\633963754.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\bhans\AppData\Local\Temp\ipykernel_20652\633963754.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [58]:
#Latitude & Longitude
train['var_max_lat'] = train['Latitude'].max() - train['Latitude']
train['var_max_long'] = train['Longitude'].max() - train['Longitude']
test['var_max_lat'] = test['Latitude'].max() - test['Latitude']
test['var_max_long'] = test['Longitude'].max() - test['Longitude']

train['lon_plus_lat'] = train['Longitude'] + train['Latitude'] 
test['lon_plus_lat'] = test['Longitude'] + test['Latitude']

C:\Users\bhans\AppData\Local\Temp\ipykernel_20652\4122607605.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\bhans\AppData\Local\Temp\ipykernel_20652\4122607605.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\bhans\AppData\Local\Temp\ipykernel_20652\4122607605.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-

In [59]:
x_train=train.drop("Number_of_Casualties",axis=1)
y_train=train["Number_of_Casualties"]

In [60]:
train_corr = train.copy()
train_corr.drop(["Accident_ID","postcode","year","country", "Time", "2nd_Road_Number","Day_of_Week","Local_Authority_(Highway)","Local_Authority_(District)","Police_Force","train_or_test"],axis=1,inplace=True)

In [61]:
#drop columns
x_train.drop(["Accident_ID","postcode","year","country", "Time", "2nd_Road_Number","Day_of_Week","Local_Authority_(Highway)","Local_Authority_(District)","Police_Force","train_or_test"],axis=1,inplace=True)
test.drop(["Accident_ID","postcode","year","country", "Time", "2nd_Road_Number","Day_of_Week","Local_Authority_(Highway)","Local_Authority_(District)","Police_Force","train_or_test","Number_of_Casualties"],axis=1,inplace=True)

C:\Users\bhans\AppData\Local\Temp\ipykernel_20652\3176366528.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [62]:
from sklearn.preprocessing import OrdinalEncoder
cols=['Speed_limit','2nd_Road_Class','1st_Road_Class','Urban_or_Rural_Area','Did_Police_Officer_Attend_Scene_of_Accident']
ord_en=OrdinalEncoder()
x_train[cols]=ord_en.fit_transform(x_train[cols])
test[cols]=ord_en.transform(test[cols])
train_corr[cols]=ord_en.transform(train_corr[cols])

#Creating dummy variables
x_train    = pd.get_dummies(x_train, drop_first=True)
test       = pd.get_dummies(test,drop_first=True)
train_corr = pd.get_dummies(train_corr, drop_first=True)

x_train.shape,test.shape,train_corr.shape

C:\Users\bhans\AppData\Local\Temp\ipykernel_20652\631355603.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



((478741, 189), (121259, 189), (478741, 190))

In [63]:
corr_matrix=train_corr.corr()

corr_matrix.sort_values('Number_of_Casualties')['Number_of_Casualties']

Police_Force_Speed_limit                                                                                     -0.071223
Police_Force_Urban_or_Rural_Area                                                                             -0.063221
Police_Force_1st_Road_Number                                                                                 -0.061910
2nd_Road_Class_Speed_limit                                                                                   -0.060796
Police_Force_2nd_Road_Class                                                                                  -0.059169
Police_Force_Road_Type                                                                                       -0.058661
density_by_Police_Force                                                                                      -0.058314
population_by_Police_Force                                                                                   -0.057700
1st_Road_Class_2nd_Road_Class_le                

# Modelling start

In [64]:
y      = train["Number_of_Casualties"]
X      = x_train.copy()
X_test = test.copy()
X.shape, X_test.shape, y.shape

((478741, 189), (121259, 189), (478741,))

In [68]:
#Basic
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
import xgboost as xgb
import lightgbm as lgb
n_folds = 10

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(y.values)
    r2= cross_val_score(model, X.values, y.values, scoring  = 'neg_root_mean_squared_error', cv = kf)
    return(r2)

model_xgb = xgb.XGBRegressor(random_state =7)
model_lgb = lgb.LGBMRegressor(random_state =7)
model_cb  = CatBoostRegressor(random_state =7, verbose=False)
model_rf  = RandomForestRegressor(random_state =7)

ModuleNotFoundError: No module named 'catboost'

In [ ]:
#score = rmsle_cv(model_lgb)
#print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
'''score = rmsle_cv(model_cb)
print("Catboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))'''

In [ ]:
'''score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))'''

In [ ]:
'''score = rmsle_cv(model_rf)
print("RF score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))'''

In [ ]:
debug=2000
rounds=90000
early_stop=200

X_1 = X.values
X_test_1 = X_test.values
target = y.values.reshape(-1,1)
verbose=debug

In [ ]:
%%time
#lgbm

train_oof = np.zeros((X.shape[0],))
test_preds = np.zeros((X_test.shape[0],))
train_oof.shape,test_preds.shape

#params = {'learning_rate': 0.017, 'max_depth': -1,'n_estimators': rounds,'metric': 'rmse',}
params = {'bootstrap_type': 'Bernoulli','n_estimators': rounds,'eval_metric': 'RMSE','learning_rate': 0.017}
#params = {'eval_metric': 'rmse',}
n_splits = 10
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X_1, target)):
    print("Fitting fold", jj+1)
    train_features = X_1[train_index]
    train_target = target[train_index]
    
    val_features = X_1[val_index]
    val_target = target[val_index]
    
    #model = LGBMRegressor(**params)
    #model = XGBRegressor(**params)
    model = CatBoostRegressor(**params)
    model.fit(train_features, train_target,
             eval_set=(val_features, val_target),
             early_stopping_rounds=early_stop,
             verbose=debug,
            )
    val_pred = model.predict(val_features)
    train_oof[val_index] = val_pred.flatten()
    test_preds += model.predict(X_test_1).flatten()/n_splits

print("\n")
print("mean_squared_error:",mean_squared_error(target,train_oof, squared=False))
print("r2_score:",r2_score(target,train_oof))

In [ ]:
def plot_feature_importance(importance,names,model_type):

    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    #Define size of bar plot
    plt.figure(figsize=(15,55))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + ' FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
    
plot_feature_importance(model.get_feature_importance(),x_train.columns,'CATBOOST')

In [ ]:
#Submit prediction
df1=pd.read_csv('test.csv')

df1["casualities"]=test_preds

Accident_risk_index=df1.groupby("postcode").casualities.sum()/df1.groupby("postcode").Accident_ID.count()

postcode=list(df1.groupby("postcode").postcode.count().index)

sub_final_df=pd.DataFrame({"postcode":postcode,"Accident_risk_index":Accident_risk_index})
sub_final_df.to_csv("cb_new_divide_by_ft_pluslat_lon_reduced.csv",index=False)
sub_final_df.shape,sub_final_df.head(10)